In [ ]:
!pip install transformers==4.18.0 torch==1.11.0 seaborn

In [ ]:
import transformers
from transformers import RobertaModel, RobertaTokenizer, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
DATA_PATH = "/data/train-balanced-sarcasm-cleaned.csv"
df = pd.read_csv(DATA_PATH,encoding="ISO-8859-1")
df = df.drop(columns=['Unnamed: 0'])
df.columns = ["target", "comment", "author","subreddit","score","ups","downs","date","created_utc","parent_comment", "text",
       "parent_cleaned_comment"]

In [ ]:
class_names = ['not_sarc', 'sarc']

In [ ]:
PRE_TRAINED_MODEL_NAME = 'roberta-base'

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
df['target']
df["target"] = pd.to_numeric(df["target"])
df.head()

In [ ]:
MAX_LEN = 60

In [ ]:
class SarcDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        text = self.df.text[item]
        target = self.df.target[item]

        encoding = self.tokenizer.encode_plus(
            str(text),
            add_special_tokens=True,
            max_length=self.max_len,
            # padding="longest",    
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "text": text,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "targets": torch.tensor(target, dtype=torch.long)
        }


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
print(len(df_train))
print(len(df_val))
print(len(df_test))

In [ ]:
df_train.reset_index(inplace=True, drop=True)
df_val.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [ ]:
def create_data_loader(df, tokenizer, MAX_LEN, batch_size):
    ds = SarcDataset(df, tokenizer, MAX_LEN)
    return DataLoader(ds, batch_size, num_workers=1)

In [ ]:
BATCH_SIZE = 200
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
class SarcRoBERTLSTMClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SarcRoBERTLSTMClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        #self.drop = nn.Dropout(p=0.3)
        self.lstm = nn.LSTM(768, 256, batch_first=True,bidirectional=True)
        self.linear = nn.Linear(256*2, n_classes)

    def forward(self, input_ids, attention_mask):
        sequence_output = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        ).last_hidden_state
        
        lstm_output, (h,c) = self.lstm(sequence_output)
        hidden = torch.cat((lstm_output[:,-1, :256],lstm_output[:,0, 256:]),dim=-1)
        linear_output = self.linear(hidden.view(-1,256*2))
        return linear_output

model = SarcRoBERTLSTMClassifier(len(class_names)).to(device)


In [ ]:
len(class_names)

In [ ]:
EPOCHS = 5

optimizer = Adam(model.parameters(), lr=2e-5, weight_decay=1e-5)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)


In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()

    losses = []
    correct_predictions = 0

    for i, d in enumerate(tqdm(data_loader)):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)


        loss.backward()
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)


In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)


In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc
# model.load_state_dict(torch.load("../input/saved-model/robertalstmpreprocess.pth").state_dict())

In [ ]:
test_acc, test_loss = eval_model(model, test_data_loader, loss_fn, device, len(df_test))
print(f"Test loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")

In [ ]:
def confusion_mat(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    predlist=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist=torch.zeros(0,dtype=torch.long, device='cpu')
    losses = []
    correct_predictions = 0
    confusion_matrix = np.zeros((len(class_names), len(class_names)))
    with torch.no_grad():
        for i, d in enumerate(tqdm(data_loader)):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            classes = d["targets"].to(device)
            #classes = classes.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            # Append batch prediction results
            predlist=torch.cat([predlist,preds.view(-1).cpu()])
            lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

    return lbllist, predlist

lbllist, predlist = confusion_mat(model, test_data_loader, loss_fn, device, len(df_test))